In [1]:
import json
from datasets import Dataset, Audio

audio = Audio(sampling_rate = 16000)

In [4]:
with open('slice-0.5.json') as fopen:
    d = json.load(fopen)
    
filtered = []
for d_ in d:
    labels = d_['labels']
    scores = d_['scores']
    if len(labels) == 1 and labels[0] == 'Speech' and scores[0] < -1.0:
        continue
    if len(labels) > 1:
        filtered_labels, filtered_scores = [], []
        for no, l in enumerate(labels):
            if l == 'Speech' and scores[no] < 1.0:
                continue
            filtered_labels.append(l)
            filtered_scores.append(scores[no])
        d_['labels'] = filtered_labels
        d_['scores'] = filtered_scores
    filtered.append(d_)
    
len(filtered), len(d)

(460888, 511857)

In [9]:
dataset = Dataset.from_list(filtered)

In [10]:
dataset = dataset.cast_column("audio_filename", audio)
dataset

Dataset({
    features: ['audio_filename', 'labels', 'scores'],
    num_rows: 460888
})

In [11]:
dataset.push_to_hub('mesolitica/AudioSet-Chunk', split = 'chunk_0.5s')

Uploading the dataset shards:   0%|          | 0/6 [00:00<?, ?it/s]

Map:   0%|          | 0/76815 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/769 [00:00<?, ?ba/s]

Map:   0%|          | 0/76815 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/769 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/769 [00:00<?, ?ba/s]

Map:   0%|          | 0/76814 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/769 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/430 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/AudioSet-Chunk/commit/0ebee957514706d5380d456335686504a2ca60d7', commit_message='Upload dataset', commit_description='', oid='0ebee957514706d5380d456335686504a2ca60d7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/AudioSet-Chunk', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/AudioSet-Chunk'), pr_revision=None, pr_num=None)

In [18]:
from collections import defaultdict
import pandas as pd

df = pd.read_csv('class_labels_indices.csv')
mapping = {}
labels = {}
rev_labels = {}
counts = defaultdict(int)
for i in range(len(df)):
    mapping[df['mid'].iloc[i]] = df['index'].iloc[i]
    labels[df['index'].iloc[i]] = df['display_name'].iloc[i]
    rev_labels[df['display_name'].iloc[i]] = df['index'].iloc[i]
    counts[df['index'].iloc[i]] = 0

In [38]:
for d in filtered:
    for l in d['labels']:
        counts[rev_labels[l]] += 1
        
for k, v in counts.items():
    print(k, labels[k], ':', v)

0 Speech : 118440
1 Male speech, man speaking : 1668
2 Female speech, woman speaking : 410
3 Child speech, kid speaking : 1150
4 Conversation : 22
5 Narration, monologue : 1588
6 Babbling : 118
7 Speech synthesizer : 596
8 Shout : 14
9 Bellow : 330
10 Whoop : 780
11 Yell : 246
12 Battle cry : 1504
13 Children shouting : 656
14 Screaming : 450
15 Whispering : 1050
16 Laughter : 1312
17 Baby laughter : 502
18 Giggle : 634
19 Snicker : 966
20 Belly laugh : 172
21 Chuckle, chortle : 836
22 Crying, sobbing : 418
23 Baby cry, infant cry : 1154
24 Whimper : 380
25 Wail, moan : 278
26 Sigh : 740
27 Singing : 16152
28 Choir : 3224
29 Yodeling : 424
30 Chant : 1446
31 Mantra : 1844
32 Male singing : 878
33 Female singing : 750
34 Child singing : 678
35 Synthetic singing : 322
36 Rapping : 1050
37 Humming : 644
38 Groan : 440
39 Grunt : 1532
40 Whistling : 1680
41 Breathing : 70
42 Wheeze : 242
43 Snoring : 470
44 Gasp : 1290
45 Pant : 598
46 Snort : 882
47 Cough : 486
48 Throat clearing : 1256
4